## Data-Oriented and ASyncIO

Reference

- https://towardsdatascience.com/data-oriented-programming-with-python-ef478c43a874/
- https://medium.com/velotio-perspectives/an-introduction-to-asynchronous-programming-in-python-af0189a88bbb
- https://realpython.com/python-async-features/

## Data-Oriented

A recap on Data-Oriented Programming by Yehonathan Sharvit (book published in 2022). Book used JavaScript and Java, in this treatment Python is used

![DOD Book Cover](gfx/dod-book-cover.png)

### Principles are language-agnostic

1. Separate code from data in a way that the code resides in functions whose behavior does not depend on data that is encapsulated in the function’s context.
2. Represent data with generic data structures
3. Data is immutable
4. Separate data schema from data representation

In [4]:
from dataclasses import dataclass

@dataclass     # <- a decorator! (a function that takes a function)
class AuthorData:
    """Class for keeping track of an author in the system"""

    first_name: str
    last_name: str
    n_books: int

# The code that deals with full name calculation is separate from the code that deals with the creation of author data.
def calculate_name(first_name: str, last_name: str):
    return f"{first_name} {last_name}"

author_data = AuthorData("Isaac", "Asimov", 500)
calculate_name(author_data.first_name, author_data.last_name)

'Isaac Asimov'